<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask Serving: Custom Train Google Object Detection to Detect ID BBox

**FIRST OF ALL: CHOOSE RUNTIME ENVIRONMENT TYPE TO BE GPU**<br>
Environment variables setup.<br>
**Tensorflow runtime version list** can be found at [here](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)

In [0]:
DEFAULT_HOME='/content'
TF_RT_VERSION='1.13'
PYTHON_VERSION='3.5'

YOUR_GCS_BUCKET='id-norm'
YOUR_PROJECT='orbital-purpose-130316'

Select the right model from [this official list](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md):

| model | dataset | datetime | notes |
| - |  - | - | - |
| ssd_inception_v2 | coco | 2018_01_28 | |
| ~~ssd_inception_v3~~ | ~~pets~~ | ~~11_06_2017~~ | |
| ssd_mobilenet_v2 | coco | 2018_03_29 | |
| faster_rcnn_resnet101 | coco | 11_06_2017 | |

In [0]:
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

MODEL_NAME = 'ssd_mobilenet_v2'
PRETRAINED_DATASET = 'coco'
PRETRAINED_TS = '2018_03_29'
PRETRAINED_MODEL_NAME = f'{MODEL_NAME}_{PRETRAINED_DATASET}_{PRETRAINED_TS}'
PIPELINE_CONFIG_NAME = f'pipeline_{MODEL_NAME}'

## Session and Environment Verification (Destination - Local)

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()


################# RE-RUN ABOVE CELLS IF NEED TO RESTART RUNTIME #################

Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [4]:
import tensorflow as tf
print(tf.__version__)
assert(tf.__version__.startswith(TF_RT_VERSION + '.')), f'tf.__version__ {tf.__version__} not matching with specified TF runtime version env variable {TF_RT_VERSION}'

1.13.1


In [5]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.16
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, wcwidth
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Install Google Object Detection API in Colab
Reference is https://colab.research.google.com/drive/1kHEQK2uk35xXZ_bzMUgLkoysJIWwznYr


### Downgrade prompt-toolkit to 1.0.15 (Destination - Local)
Run this **ONLY** if the Installation not Working

In [0]:
# !pip install 'prompt-toolkit==1.0.15'

### Google Object Detection API Installation (Destination - Local)

In [7]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
![ ! -e {DEFAULT_HOME}/models ] && git clone --depth=1 --quiet https://github.com/tensorflow/models.git {DEFAULT_HOME}/models
!ls {DEFAULT_HOME}/models

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [8]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')
!pwd

/content/models/research


*From Wikipedia ...*: 

**protocol buffers** are a language-neutral, platform-neutral extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler. 

You define how you want your data to be structured once, then you can **use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages**.

Remember **.proto defines structured data** and **protoc generates the source code** the serailize/de-serialize.

In [9]:
!protoc object_detection/protos/*.proto --python_out=.
# !ls object_detection/protos/*.proto
# !cat object_detection/protos/anchor_generator.proto
!ls {DEFAULT_HOME}/models/research/object_detection/builders/anchor*

/content/models/research/object_detection/builders/anchor_generator_builder.py
/content/models/research/object_detection/builders/anchor_generator_builder_test.py


#### Add Google Object Detection API into System Path

In [0]:
import sys
sys.path.append(f'{DEFAULT_HOME}/models/research')
sys.path.append(f'{DEFAULT_HOME}/models/research/slim')

Note that ! calls out to a shell (in a **NEW** process), while % affects the **SAME** process associated with the notebook.

Since we append pathes to sys.path, we **HAVE TO** use % command to run the Python

Also it is **IMPORTANT** to have **%matplotlib inline** otherwise %run model_builder_test.py will **cause function attribute error** when accessing matplotlib.pyplot attributes from **iPython's run_line_magic** 

In [0]:
# !find . -name 'inception*' -print
%matplotlib inline

In [14]:
# If see the error 'function' object has no attribute 'called', just run the %matplotlib cell and this cell AGAIN 
%run object_detection/builders/model_builder_test.py

import os
os.chdir(f'{DEFAULT_HOME}')

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.099s

OK (skipped=1)


## Git Sync for any Change in colab-god-idclass 

In [15]:
![ -e {DEFAULT_HOME}/colab-god-idclass ] && git -C {DEFAULT_HOME}/colab-god-idclass pull
![ ! -e {DEFAULT_HOME}/colab-god-idclass ] && git clone --depth=1 https://github.com/hailusong/colab-god-idclass.git {DEFAULT_HOME}/colab-god-idclass

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/hailusong/colab-god-idclass
   f687fc1..68a2859  master     -> origin/master
Updating f687fc1..68a2859
Fast-forward
 src/idservice.py | 8 ++++----
 1 file changed, 4 insertions(+), 4 deletions(-)


### Checking Your Google Cloud Storage Bucket

In [16]:
!gsutil ls gs://{YOUR_GCS_BUCKET}/data_{MODEL_NAME}
!gsutil ls gs://{YOUR_GCS_BUCKET}/generated

gs://id-norm/data_ssd_mobilenet_v2/label_map.pbtxt
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.data-00000-of-00001
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.index
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.meta
gs://id-norm/data_ssd_mobilenet_v2/pipeline_ssd_mobilenet_v2_processed.config
gs://id-norm/data_ssd_mobilenet_v2/test.record
gs://id-norm/data_ssd_mobilenet_v2/train.record
gs://id-norm/generated/bbox-train-non-id1.csv
gs://id-norm/generated/bbox-train-non-id2.csv
gs://id-norm/generated/bbox-train-non-id3.csv
gs://id-norm/generated/bbox-train-on-dl.csv
gs://id-norm/generated/bbox-train-on-hc.csv
gs://id-norm/generated/bbox-valid-non-id1.csv
gs://id-norm/generated/bbox-valid-non-id2.csv
gs://id-norm/generated/bbox-valid-non-id3.csv
gs://id-norm/generated/bbox-valid-on-dl.csv
gs://id-norm/generated/bbox-valid-on-hc.csv
gs://id-norm/generated/pnts-train-non-id1.csv
gs://id-norm/generated/pnts-train-non-id2.csv
gs://id-norm/generated/pnts-train-non-id3.csv
gs://id-norm/gene

### Download trained Dlib key points model

In [0]:
NU=0.1
TREE_DEPTH=4
CASCADE_DEPTH=15
FEATURE_POOL_SIZE=800
NUM_TEST_SPLITS=200
OVERSAMPLING_AMT=10

In [0]:
id_keypoints_fname = f'id_keypoints_dlib_nu{NU}td{TREE_DEPTH}cd{CASCADE_DEPTH}fp{FEATURE_POOL_SIZE}nt{NUM_TEST_SPLITS}oa{OVERSAMPLING_AMT}.dat'
id_keypoints_dat = f'{DEFAULT_HOME}/id_keypoints_dlib.dat'

![ ! -e {id_keypoints_dat} ] && gsutil cp gs://{YOUR_GCS_BUCKET}/{id_keypoints_fname} {id_keypoints_dat}

## Run Flask

### Setup Ngrok

In [19]:
!pip install ujson
!pip install lz4
!pip install py-lz4framed
!pip install pillow

In [20]:
!pip install flask
!pip install flask-ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-18 17:29:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.40.183, 52.3.53.115, 52.72.250.2, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.232.40.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.5’

ngrok-stable-linux- 100%[===================>]  14.28M  40.9MB/s    in 0.3s    

2019-04-18 17:29:17 (40.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.5’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


### Explicitly Import Flask
Otherwise there could be 'Cannot Import Flask' error when running Flask.py

In [0]:
from flask import Flask

### Try Inference First

In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

def preview(pil_im:Image, rect:list, markers_x:list, markers_y:list):
    """
    rect:
        (x, y, width, height)
    
    markers_x/y:
        (x0, x1, ...)
        (y0, y1, ...)
    """
    im = np.array(pil_im, dtype=np.uint8)

    # create figure and axes
    fig ,ax = plt.subplots(1)

    # display the image
    ax.imshow(im)

    # create a Rectangle patch
    rect = patches.Rectangle(rect[:2], *rect[2:], linewidth=1, edgecolor='r', facecolor='none')

    # add the patch to the Axes
    ax.add_patch(rect)

    # markers
#     plt.annotate('25, 50', xy=(25, 50), xycoords='data',
#              xytext=(0.5, 0.5), textcoords='figure fraction',
#              arrowprops=dict(arrowstyle="->"))
    plt.scatter(markers_x, markers_y, c='red', marker='o')

    plt.show()

In [23]:
import sys
sys.path.append(f'{DEFAULT_HOME}/colab-god-idclass/src')

import idservice

# download some test images
![ ! -e {DEFAULT_HOME}/Capture0.PNG ] && gsutil cp gs://{YOUR_GCS_BUCKET}/test_images/Capture0.PNG {DEFAULT_HOME}/.

1.13.1


In [0]:
# some snippet from idservice._inference
# ------------------------------------------
# image_np = idservice.load_image_into_numpy_array(im)

# # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
# image_np_expanded = np.expand_dims(image_np, axis=0)

# # Actual detection.
# output_dict = idservice.run_inference_for_single_image(image_np, idservice.detection_graph)

# # confidence threshold is 0.8
# indic = np.argmax(output_dict['detection_scores'])
# confidence = output_dict['detection_scores'][indic]

im = Image.open(f'{DEFAULT_HOME}/Capture0.PNG')
confidence, bboxes, bboxes_abs, labels, pnts = idservice._inference(im)

markers_x, markers_y = zip(*pnts)

if confidence >= 0.8:
    preview(im, bboxes_abs, markers_x, markers_y)
else:
    print(f'{confidence} is less than threshold 0.8')

### Run Flask to Serve Inference
***BEFORE RUNNING THIS MAKE SURE THE MODEL_NAME IS SET TO PROPER VALUE IN FLASK.PY***

In [0]:
# Start ngrok tunneling process in the background
get_ipython().system_raw('./ngrok http 5000 &')
# !curl -s http://localhost:4040/api/tunnels | python3 -c \
#        "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# flask_ngrok_example.py
# from flask import Flask
# from flask_ngrok import run_with_ngrok
# 
# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run
# 
# @app.route("/")
# def hello():
#     return "Hello World!"
# 
# if __name__ == '__main__':
#     app.run()

# start up flask and connect with ngrok
%run {DEFAULT_HOME}/colab-god-idclass/src/flask.py